# Carga de librerias y datos

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia
!nvidia-smi

chmod: cannot access './kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in .. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
dtr= pd.read_csv("train.csv")

In [ ]:
dtr.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# PREPROCESAMIENTO

## DATOS BOOLEANOS

In [ ]:
def mask(x):
  if x=="Si":
    return bool(True)
  elif x=="No":
    return bool(False)
  else:
    return
#Se define una mascara para que los datos queden en el tipo adecuado

In [ ]:
for i in ['FAMI_TIENEINTERNET','FAMI_TIENELAVADORA','FAMI_TIENEAUTOMOVIL','ESTU_PAGOMATRICULAPROPIO','FAMI_TIENECOMPUTADOR','FAMI_TIENEINTERNET.1']:
  dtr[i]=dtr[i].apply(mask)

In [ ]:
dtr['FAMI_TIENEINTERNET'].value_counts(dropna=False)


,count
FAMI_TIENEINTERNET,
True,592514
False,73357
None,26629


In [ ]:
dtr['FAMI_TIENEINTERNET.1'].value_counts(dropna=False)

,count
FAMI_TIENEINTERNET.1,
True,592514
False,73357
None,26629


In [ ]:
del dtr['FAMI_TIENEINTERNET.1'] #Observamos que es una variable repetida por lo cual borramos una de las columnas

In [ ]:
def mask1(x):
  if x=="S":
    return bool(True)
  elif x=="N":
    return bool(False)
  else:
    return
#Mascara para variable particular

In [ ]:
dtr['ESTU_PRIVADO_LIBERTAD']=dtr['ESTU_PRIVADO_LIBERTAD'].apply(mask1)

In [ ]:
bool_var= ['FAMI_TIENEINTERNET','FAMI_TIENELAVADORA','FAMI_TIENEAUTOMOVIL','ESTU_PAGOMATRICULAPROPIO','FAMI_TIENECOMPUTADOR','ESTU_PRIVADO_LIBERTAD']

In [ ]:
for col in bool_var:
  dtr[col]=dtr[col].fillna(dtr[col].mode(dropna=True)[0])
  #Al ser datos booleanos, y la cantidad de faltantes no muy alta, se decide llenar los datos faltantes con la moda de cada una de las columnas

/tmp/ipython-input-24-3536273753.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dtr[col]=dtr[col].fillna(dtr[col].mode(dropna=True)[0])


## Datos categoricos mapeables

Hay una buena cantidad de columnas que tienen una estructura ordenada por lo cual para simplicidad de los datos podriamos mapearlos en integrers ordenadas, en ciertos casos se toma la decisión de llenar los valores faltantes con 0, la media, o una categoría nueva segun tenga sentido.

In [ ]:
dtr['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts(dropna=False)

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
Entre 1 millón y menos de 2.5 millones,204048
Entre 2.5 millones y menos de 4 millones,127430
Menos de 500 mil,80263
Entre 500 mil y menos de 1 millón,78704
Entre 4 millones y menos de 5.5 millones,69736
Más de 7 millones,68014
Entre 5.5 millones y menos de 7 millones,38490
No pagó matrícula,19528
NaN,6287


En este caso podemos ver que la variable tiene una estructura continua, los datos faltantes son bastante pocos, y se nota que no tienen ningun significado por si solos por lo cual es conveniente ponerlos en el valor entero más cercano a la media

In [ ]:
mapa_matricula = {
    'Menos de 500 mil': 1,
    'Entre 500 mil y menos de 1 millón': 2,
    'Entre 1 millón y menos de 2.5 millones': 3,
    'Entre 2.5 millones y menos de 4 millones': 4,
    'Entre 4 millones y menos de 5.5 millones': 5,
    'Entre 5.5 millones y menos de 7 millones': 6,
    'Más de 7 millones': 7,
    'No pagó matrícula': 0,
}
dtr['ESTU_VALORMATRICULAUNIVERSIDAD'] = dtr['ESTU_VALORMATRICULAUNIVERSIDAD'].map(mapa_matricula)

In [ ]:
dtr['ESTU_VALORMATRICULAUNIVERSIDAD']=dtr['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(round(dtr['ESTU_VALORMATRICULAUNIVERSIDAD'].mean()))

In [ ]:
dtr['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts(dropna=False)

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
3.0,204048
4.0,133717
1.0,80263
2.0,78704
5.0,69736
7.0,68014
6.0,38490
0.0,19528


In [ ]:
dtr['ESTU_HORASSEMANATRABAJA'].value_counts(dropna=False)

,count
ESTU_HORASSEMANATRABAJA,
Más de 30 horas,249352
0,116550
Entre 11 y 20 horas,115857
Entre 21 y 30 horas,92693
Menos de 10 horas,87191
NaN,30857


En este caso aplicamos el mismo metodo

In [ ]:
mapa_trabajo = {
    '0': 0,
    'Menos de 10 horas': 1,
    'Entre 11 y 20 horas': 2,
    'Entre 21 y 30 horas': 3,
    'Más de 30 horas': 4
}

dtr['ESTU_HORASSEMANATRABAJA'] = dtr['ESTU_HORASSEMANATRABAJA'].map(mapa_trabajo)
dtr['ESTU_HORASSEMANATRABAJA'] = dtr['ESTU_HORASSEMANATRABAJA'].fillna(round(dtr['ESTU_HORASSEMANATRABAJA'].mean()))

In [ ]:
dtr['ESTU_HORASSEMANATRABAJA'].value_counts(dropna=False)

,count
ESTU_HORASSEMANATRABAJA,
4.0,249352
2.0,146714
0.0,116550
3.0,92693
1.0,87191


In [ ]:
dtr['FAMI_ESTRATOVIVIENDA'].value_counts(dropna=False)

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,232671
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
NaN,32137
Estrato 5,23608
Estrato 6,12605
Sin Estrato,3289


En este caso los datos faltantes son mayores que varias categorias, por lo cual se supondra que es otra categoría difrente, en este caso no sabe no responde, esta categoría a pesar de que se puede mapear, al tener datos de este tipo la dejaremos en string

In [ ]:
dtr['FAMI_ESTRATOVIVIENDA']=dtr['FAMI_ESTRATOVIVIENDA'].fillna('ns/nr')
dtr['FAMI_ESTRATOVIVIENDA'].value_counts(dropna=False)

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,232671
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
ns/nr,32137
Estrato 5,23608
Estrato 6,12605
Sin Estrato,3289


In [ ]:
dtr['FAMI_EDUCACIONMADRE'].value_counts(dropna=False)

,count
FAMI_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,141744
Primaria incompleta,99420
Técnica o tecnológica completa,89542
Educación profesional completa,85326
Secundaria (Bachillerato) incompleta,81012
Primaria completa,56125
Postgrado,46246
Técnica o tecnológica incompleta,27533
NaN,23664


In [ ]:
dtr['FAMI_EDUCACIONPADRE'].value_counts(dropna=False)

,count
FAMI_EDUCACIONPADRE,
Secundaria (Bachillerato) completa,128289
Primaria incompleta,125675
Educación profesional completa,83117
Secundaria (Bachillerato) incompleta,71654
Técnica o tecnológica completa,62995
Primaria completa,55958
Postgrado,44169
Educación profesional incompleta,27084
NaN,23178


Nuevamente en estos dos casos aplica el caso anterior, los datos faltantes son más grandes que algunas categorias por lo cual se le dará su propia categoria en desconocido.

In [ ]:
for col in ['FAMI_EDUCACIONMADRE','FAMI_EDUCACIONPADRE']:
  dtr[col]=dtr[col].fillna('desconocido')

In [ ]:
dtr.isnull().any()

,0
ID,False
PERIODO,False
ESTU_PRGM_ACADEMICO,False
ESTU_PRGM_DEPARTAMENTO,False
ESTU_VALORMATRICULAUNIVERSIDAD,False
ESTU_HORASSEMANATRABAJA,False
FAMI_ESTRATOVIVIENDA,False
FAMI_TIENEINTERNET,False
FAMI_EDUCACIONPADRE,False
FAMI_TIENELAVADORA,False


In [ ]:
Var_categoricas=['FAMI_EDUCACIONMADRE','FAMI_EDUCACIONPADRE','FAMI_ESTRATOVIVIENDA','ESTU_PRGM_ACADEMICO','ESTU_PRGM_DEPARTAMENTO']
for col in Var_categoricas:
  dtr[col]=dtr[col].astype(str)